In [45]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
# from langchain_core.messages import SystemMessage, HumanMessage
# import operator
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [46]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


In [47]:
load_dotenv()
llm = ChatOpenAI(api_key=api_key)  # Replace with your OpenAI API key

In [48]:
class JokeState(TypedDict):
    
    topic: str
    joke: str
    explanation: str

In [49]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {"joke": response}

In [50]:
def generate_explanation(state: JokeState):
    prompt = f'Write an explanation for the joke: {state["joke"]}'
    response = llm.invoke(prompt).content

    return {"explanation": response}

In [51]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [52]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({"topic":"pizza"}, config=config1)

{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': 'This joke plays on the idea of a pizza having "too many toppings" and feeling overwhelmed by the pressure of carrying so much weight. In the context of therapy, people often seek help when they feel overwhelmed or unable to handle the pressures of life, so the joke imagines a scenario where a pizza would do the same because it has too many toppings.'}

In [53]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea of a pizza having "too many toppings" and feeling overwhelmed by the pressure of carrying so much weight. In the context of therapy, people often seek help when they feel overwhelmed or unable to handle the pressures of life, so the joke imagines a scenario where a pizza would do the same because it has too many toppings.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-8d82-6bfd-8002-e8117dc3ae68'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-13T20:23:02.070956+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-7f41-67dd-8001-0aaf424fc013'}}, tasks=(), interrupts=())

In [54]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke plays on the idea of a pizza having "too many toppings" and feeling overwhelmed by the pressure of carrying so much weight. In the context of therapy, people often seek help when they feel overwhelmed or unable to handle the pressures of life, so the joke imagines a scenario where a pizza would do the same because it has too many toppings.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-8d82-6bfd-8002-e8117dc3ae68'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-13T20:23:02.070956+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-7f41-67dd-8001-0aaf424fc013'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the the

In [55]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({"topic":"pasta"}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta go to the party? Because it heard it was going to be a "shell" of a good time!',
 'explanation': 'This joke plays on the pun of the word "shell," which can refer to both a type of pasta (like shell-shaped noodles) and a term meaning a lot of fun or a great time. The pasta went to the party because it heard it was going to be a "shell" of a good time, meaning it was going to be a really fun and enjoyable event. This joke relies on the play on words to create humor and bring a light-hearted tone to the situation.'}

In [56]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta go to the party? Because it heard it was going to be a "shell" of a good time!', 'explanation': 'This joke plays on the pun of the word "shell," which can refer to both a type of pasta (like shell-shaped noodles) and a term meaning a lot of fun or a great time. The pasta went to the party because it heard it was going to be a "shell" of a good time, meaning it was going to be a really fun and enjoyable event. This joke relies on the play on words to create humor and bring a light-hearted tone to the situation.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-a118-606c-8002-2b3a2932b6ed'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-13T20:23:04.124408+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-936d-6454-8001-a6b4ca25a6ea'}}, tasks=(), interrupts=())

# TIME TRAVEL

In [57]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [58]:
# workflow.invoke({}, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"}})
workflow.invoke(
    {"topic": "pizza"},
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"
        }
    }
)


{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little saucy!',
 'explanation': 'This joke plays on the term "saucy", which can mean both having a cheeky or flirtatious attitude and containing a lot of sauce. In this case, the pizza went to the doctor because it was feeling "saucy" in the sense that it had a lot of sauce on it, not in a cheeky way.'}

In [59]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little saucy!', 'explanation': 'This joke plays on the term "saucy", which can mean both having a cheeky or flirtatious attitude and containing a lot of sauce. In this case, the pizza went to the doctor because it was feeling "saucy" in the sense that it had a lot of sauce on it, not in a cheeky way.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-bb8b-6308-8002-682d8d0e300e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-13T20:23:06.897862+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-a9a3-64f6-8001-608021002334'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little saucy!'}, next=('generate_explanation',), config={'configurable': {'thread_i

# Updating State

In [60]:
workflow.update_state({"configurable": {"thread_id":"1", "checkpoint_id":"1f0f0b64-6df9-6d1f-8000-7e496f33741c", "checkpoint_ns":""}}, {"topic":"samosa"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0f0bda-bc2a-6ffe-8000-b1a297077ceb'}}

In [61]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-bc2a-6ffe-8000-b1a297077ceb'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-13T20:23:06.963330+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0b64-6df9-6d1f-8000-7e496f33741c'}}, tasks=(PregelTask(id='5baf7e3e-b113-e41e-17da-e81a5ce57e3b', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it was feeling a little saucy!', 'explanation': 'This joke plays on the term "saucy", which can mean both having a cheeky or flirtatious attitude and containing a lot of sauce. In this case, the pizza went to the doctor because it was feeling "saucy" in the sense that it had a lot of s

In [62]:
workflow.invoke(
    {"topic": "samosa"},
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f0f0b7f-493c-6ced-8001-1477af215b28"
        }
    }
)

{'topic': 'samosa',
 'joke': 'Why did the samosa go to the comedy club? Because it wanted to be a little more punjabi-c!',
 'explanation': 'This joke plays on the word "punny" which means clever or witty, and "Punjabi" which refers to the Indian state of Punjab. In this joke, the samosa (a popular Indian snack) wants to be more "punjabi-c" or "punny", so it goes to a comedy club in hopes of becoming more humorous. It\'s a play on words that combines the idea of being clever with a reference to Punjabi culture.'}

In [63]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the comedy club? Because it wanted to be a little more punjabi-c!', 'explanation': 'This joke plays on the word "punny" which means clever or witty, and "Punjabi" which refers to the Indian state of Punjab. In this joke, the samosa (a popular Indian snack) wants to be more "punjabi-c" or "punny", so it goes to a comedy club in hopes of becoming more humorous. It\'s a play on words that combines the idea of being clever with a reference to Punjabi culture.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-d0e1-6522-8002-2f608e29d9cf'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-13T20:23:09.135155+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0bda-c573-6abd-8001-a2e09bb73f19'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa g